# 连接数据库，初始化数据表

In [37]:
import MySQLdb
import requests
import json
import datetime
import tushare as ts
from bs4 import BeautifulSoup

class MySQLManager:
    def __init__(self):
        # 打开数据库连接
        self.db = MySQLdb.connect("192.168.56.103", "guest", "welcome", "guest", charset='utf8' )
        self.createTableZTSTOCKS()
    
    def close(self):
        # 关闭数据库连接
        self.db.close()
        
    def createTableZTSTOCKS(self):
        # 使用cursor()方法获取操作游标 
        cursor = self.db.cursor()

        # 创建数据表SQL语句
        sql = """CREATE TABLE ZTSTOCKS (
                 DATE date,
                 STOCK_CODE CHAR(6) NOT NULL,
                 STOCK_NAME VARCHAR(20),
                 NOW_PRICE FLOAT,
                 PRICE_LIMIT FLOAT,
                 FCB FLOAT,
                 FLB FLOAT,
                 FB_MONEY FLOAT,
                 FIRST_ZT_TIME TIME,
                 LAST_ZT_TIME TIME,
                 OPEN_TIME INT,
                 ZHEN_FU FLOAT,
                 ZT_FORCE FLOAT)"""
        try:
            cursor.execute(sql)
        except:
            self.db.rollback()
        cursor.close()
        
    def insertIntoZTSTOCKS(self, records):
        cursor = self.db.cursor()
        #try:
        for item in records:
            sql = "INSERT INTO ZTSTOCKS VALUES('%s','%s','%s',%s,%s,%s,%s,%s,'%s','%s',%s,%s,%s)" % tuple(item)
            #print(sql)
            cursor.execute(sql)
        self.db.commit()
        #except:
            #self.db.rollback()
        cursor.close()
    
    # 返回数据库中最大的日期，返回值为datetime类型
    def maxDate(self):
        cursor = self.db.cursor()
        cursor.execute("SELECT max(date) from ZTSTOCKS")
        maxdate = cursor.fetchone()[0]
        if maxdate is None:
            maxdate = datetime.date.today() - datetime.timedelta(days=60)
        return maxdate

# DataCrawler类，用来从相关网站上爬取财经数据

In [38]:
class DataCrawler:
    def __init__(self):
        pass
    
    # 获取当日涨停板列表，参数：日期（20180507）
    def get_ztb_list(self, date):
        url = "http://home.flashdata2.jrj.com.cn/limitStatistic/ztForce/%s.js" % date
        headers = {'user-agent':'User-Agent: Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36'}
        resp = requests.get(url, headers=headers)
        if resp.status_code != 200:
            return []
        raw_text = resp.text
        jsonstr = raw_text[raw_text.find('[') : (raw_text.rfind(']') + 1)]
        data = json.loads(jsonstr)
        return data
    
    # 获取过去连续N天的涨停板列表
    def crawl_ztb_of_last_n_days(self, n):
        ret = []
        date = datetime.date.today()
        count = 0
        while count < n:
            ztb_list = self.get_ztb_list(date.strftime("%Y%m%d"))
            for item in ztb_list:
                item.insert(0, date.isoformat())
                ret.append(item)
            date = date - datetime.timedelta(days=1)
            count += 1
        return ret

# 将数据库中的数据更新至今天
def updateStockDatabase():
    mysqlManager = MySQLManager()
    maxDateInDB = mysqlManager.maxDate()
    deltaDate = datetime.date.today() - maxDateInDB

    dc = DataCrawler()
    result = dc.crawl_ztb_of_last_n_days(deltaDate.days)
    if len(result) != 0:
        print("获取到%d条新的涨停板数据" % len(result))
        mysqlManager.insertIntoZTSTOCKS(result)
        print("更新成功！上次更新日期为%s" % mysqlManager.maxDate())
    else:
        print("未找到新的涨停板数据，上次更新日期为%s" % mysqlManager.maxDate())
    mysqlManager.close()

updateStockDatabase()

获取到1956条新的涨停板数据
更新成功！上次更新日期为2018-05-18
